In [4]:
import requests
from bs4 import BeautifulSoup

def getNames(domain_name):
    url = f"https://crt.sh/?q={domain_name}&exclude=expired&group=none"
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html')
        
        return soup




In [5]:
soup = getNames("google.com")

# find all the <tr> in soup and put them in a list

tr = soup.find_all("tr")

print(tr[1])


<tr>
<th class="outer">Certificates</th>
<td class="outer">
<table>
<tr>
<th>
<a href="?q=google.com&amp;exclude=expired&amp;dir=v&amp;sort=0&amp;exclude=expired&amp;group=none">crt.sh ID</a>
</th>
<th style="white-space:nowrap">
       <a href="?q=google.com&amp;exclude=expired&amp;dir=v&amp;sort=1&amp;exclude=expired&amp;group=none">Logged At</a> 
 ⇧    </th>
<th style="white-space:nowrap"><a href="?q=google.com&amp;exclude=expired&amp;dir=v&amp;sort=2&amp;exclude=expired&amp;group=none">Not Before</a>
</th>
<th style="white-space:nowrap"><a href="?q=google.com&amp;exclude=expired&amp;dir=v&amp;sort=4&amp;exclude=expired&amp;group=none">Not After</a>
</th>
<th>Common Name</th>
<th>Matching Identities</th>
<th>
<a href="?q=google.com&amp;exclude=expired&amp;dir=v&amp;sort=3&amp;exclude=expired&amp;group=none">Issuer Name</a>
</th>
</tr>
<tr>
<td style="text-align:center"><a href="?id=13655732793">13655732793</a></td>
<td style="text-align:center;white-space:nowrap">2024-07-08</td>
<td

In [7]:
tr.pop(0)
tr.pop(0)
tr.pop(0)


<tr>
<td style="text-align:center"><a href="?id=13655643499">13655643499</a></td>
<td style="text-align:center;white-space:nowrap">2024-07-08</td>
<td style="text-align:center;white-space:nowrap">2024-07-08</td>
<td style="text-align:center;white-space:nowrap">2024-09-30</td>
<td>*.apis.google.com</td>
<td>*.apis.google.com<br/>apis.google.com</td>
<td><a href="?caid=286237" style="white-space:normal">C=US, O=Google Trust Services, CN=WE2</a></td>
</tr>

In [26]:
a = tr[8].find_all("td")

domaine = a[4].text
associated = a[5].text

print(f"Domaine : {domaine} \nAssociated : {associated}\na is :\n{[i.text for i in a]}")

Domaine : *.webapp.google.com 
Associated : *.webapp.google.com
a is :
['13582142151', '2024-07-01', '2024-07-01', '2024-09-23', '*.webapp.google.com', '*.webapp.google.com', 'C=US, O=Google Trust Services, CN=WE2']


In [31]:
from bs4 import BeautifulSoup

# Function to replace <br> tags with a separator and extract text
def extract_text_with_separator(element, separator=", "):
    html = str(element)
    # Replace <br> and <br/> with the separator
    html_replaced = html.replace("<br>", separator).replace("<br/>", separator)
    # Parse the modified HTML string
    soup = BeautifulSoup(html_replaced, "html.parser")
    # Extract and return the text
    return soup.get_text()

# Initialize an empty dictionary to store domains and associated domains
domains_dict = {}

# Loop through each <tr> element
for tr_element in tr:
    td_elements = tr_element.find_all("td")
    
    # Ensure there are enough <td> elements to avoid IndexError
    if len(td_elements) > 5:
        domain = td_elements[4].text
        # Manually handle <br> tags in the 5th <td>
        associated = extract_text_with_separator(td_elements[5], ", ")
        
        # Store the domain and associated domain in the dictionary
        domains_dict[domain] = associated

# Now you have a dictionary with unique domains and their associated domains
print(domains_dict)

{'accounts.google.com': 'accounts.google.com', 'checkout.google.com': 'checkout.google.com', 'classroom.google.com': '*.classroom.google.com, classroom.google.com', 'misc-sni.google.com': '*.aiplatform-notebook.cloud.google.com, *.aiplatform-training.cloud.google.com, *.backupdr-autopush.cloud.google.com, *.backupdr.cloud.google.com, *.backupdr-dev.cloud.google.com, *.backupdr-sandbox.cloud.google.com, *.backupdr-staging.cloud.google.com, colab.research.google.com, *.composer.cloud.google.com, *.composer-dev.cloud.google.com, *.composer-qa.cloud.google.com, *.composer-staging.cloud.google.com, console.eu.cloud.google.com, *.datafusion-api.cloud.google.com, *.datafusion-api-dev.cloud.google.com, *.datafusion-api-staging.cloud.google.com, *.datafusion.cloud.google.com, *.datafusion-dev.cloud.google.com, *.datafusion-staging.cloud.google.com, *.datalab.cloud.google.com, *.dataplex.cloud.google.com, *.dataplex-dev.cloud.google.com, *.dataplex-staging.cloud.google.com, *.dataproc.cloud.goog

In [33]:
# save domais_dict as csv
import pandas as pd

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(domains_dict.items()), columns=["Domain", "Associated Domains"])
# Save the DataFrame to a CSV file
df.to_csv("./../data/domains.csv", index=False)